# I. US County Race Population

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime  


def diagnose_Rate(T, **kwargs):
    info_list = []
    for col in T.columns:
        if col in ['Rate_Black', 'Rate_White', 'Rate_Parity', 'FullRate_Black', 'FullRate_White', 'FullRate_Parity']:
            d = {'name': col}
            for k, v in kwargs.items():
                d[k] = v
                
            for k,v in T[col].describe().to_dict().items():
                d[k] = v
            info_list.append(d)

    Report = pd.DataFrame(info_list)
    return Report



path = 'DataMerge/Data/C-AnnualCountyPopulation/US_COUNTY_POPULATION.csv'
US_COUNTY_POPULATION = pd.read_csv(path)
US_COUNTY_POPULATION.head()

,GEOID,State,County,Geographic Area Name,Total_Whole,Total_White,Total_Black,Total_AIndA,Total_Asian,Total_WhiteC,Total_BlackC,Total_AIndAC,Total_AsianC
0,1001,Alabama,Autauga,"Autauga, Alabama",55869,42607,11237,266,656,43563,11784,575,950
1,1003,Alabama,Baldwin,"Baldwin, Alabama",223234,195198,19593,1742,2380,199103,21354,3603,3351
2,1005,Alabama,Barbour,"Barbour, Alabama",24686,12134,11906,170,116,12380,12117,297,152
3,1007,Alabama,Bibb,"Bibb, Alabama",22394,17191,4763,103,48,17420,4907,207,97
4,1009,Alabama,Blount,"Blount, Alabama",57826,55412,978,370,185,56167,1276,820,312


In [2]:

STATE_VACCINE = {}


In [3]:
print("Enter the date of collecting data (Today's data in YYYY-MM-DD):")
Today_Date = input()

assert Today_Date == str(datetime.now().date())

print('Hello, ' + Today_Date)


Enter the date of collecting data (Today's data in YYYY-MM-DD):
2021-05-20
Hello, 2021-05-20


# II. API States

## [!] A. California* (Done)


https://covid19.ca.gov/vaccines/#California-vaccines-dashboard


https://data.ca.gov/dataset/covid-19-vaccine-progress-dashboard-data 


https://data.ca.gov/dataset/covid-19-vaccine-progress-dashboard-data/resource/c020ef6b-2116-4775-b11d-9df2875096ab # click here



Statewide COVID-19 Vaccines Administered By CountyCSV


Direct Vaccine Rate 

No Fully Rate information

In [5]:
'CountyVaccine/TMP/May20/California/covid19vaccinesbycounty.csv'

'CountyVaccine/TMP/May20/California/covid19vaccinesbycounty (4).csv'

In [6]:
import os
from datetime import datetime 
import numpy as np
import requests
import time
 
Date = Today_Date
    
while True:
    Path = input('Enter California Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)



# CalInfo: county total vaccine number each date
CalInfo = pd.read_csv(Path)
CalInfo['administered_date'] = pd.to_datetime(CalInfo['administered_date'])
CalInfo = CalInfo[CalInfo['administered_date'] == Date].reset_index(drop = True)


while len(CalInfo) == 0:
    print('There is no information for current Date:', Date)
    Date = input('Please Enter a new One:')
    CalInfo = pd.read_csv(Path)
    CalInfo = CalInfo[CalInfo['administered_date'] == Date].reset_index(drop = True)
    

print('We get California Information for Date:', Date)
CalInfo = CalInfo[['county', 'california_flag', 'cumulative_at_least_one_dose']]
CalInfo['Geographic Area Name'] = CalInfo['county'] + ', ' + CalInfo['california_flag']
print(CalInfo.shape)


CalInfo

Enter California Manually collected Data:  CountyVaccine/TMP/May20/California/covid19vaccinesbycounty (4).csv
Wrong file path...
Enter California Manually collected Data:  CountyVaccine/TMP/May20/California/covid19vaccinesbycounty.csv
File Exists!
Enter "Yes" if the file contains the information of the date you wanted:  Yes

The Path we have:
CountyVaccine/TMP/May20/California/covid19vaccinesbycounty.csv
There is no information for current Date: 2021-05-20
Please Enter a new One:2021-05-19
We get California Information for Date: 2021-05-19
(62, 4)


,county,california_flag,cumulative_at_least_one_dose,Geographic Area Name
0,All CA Counties,NaN,20328003,NaN
1,All CA and Non-CA Counties,NaN,20733486,NaN
2,Outside California,Not in California,400563,"Outside California, Not in California"
3,Unknown,Not in California,4919,"Unknown, Not in California"
4,Alameda,California,1032515,"Alameda, California"
...,...,...,...,...
57,Tulare,California,164270,"Tulare, California"
58,Tuolumne,California,22749,"Tuolumne, California"
59,Ventura,California,451254,"Ventura, California"
60,Yolo,California,119605,"Yolo, California"


In [7]:

def get_California_Raw_and_postprocess(US_COUNTY_POPULATION):
    state = 'California'
    ca_county_list = [i.lower().replace(' ','') for i in US_COUNTY_POPULATION[US_COUNTY_POPULATION['State'] == 'California']['County'].values]

    L = []
    error_list = []
    for i in ca_county_list:

        api_url = 'https://files.covid19.ca.gov/data/vaccine-equity/race-ethnicity/vaccines_by_race_ethnicity_{}.json'.format(i)

        resp = requests.get(api_url)# .to_json()
        data = resp.json() # resp

        try:

            d = {}
            d['County'] = data['meta']['REGION']
            d['Latest_Date'] = data['meta']['LATEST_ADMIN_DATE']
            for i in data['data']:
                d[i['CATEGORY']] = i['METRIC_VALUE']
            L.append(d)

        except:
            print(i)
            error_list.append(i)

    CA_County_Vaccine_Race = pd.DataFrame(L)
    
    return CA_County_Vaccine_Race





def standardize_California(CA_County_Vaccine_Race, US_COUNTY_POPULATION):
    
    # IL_County_Vaccine_Race = pd.DataFrame(L)
    state = 'California'

    new_L = []


    for idx, row in CA_County_Vaccine_Race.iterrows():
        d = row.to_dict()

        new_d = {}


        new_d['State'] = state
        new_d['County'] = d['County']
        
        RenameCounties = {
            # 'Mckean': 'McKean'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']



        # new_d['Numb_White'] = np.nan
        new_d['Rate_White_V'] = d['White']

        # new_d['Numb_Black'] = np.nan
        new_d['Rate_Black_V'] = d['Black']

        # new_d['Numb_Asian'] = np.nan
        new_d['Rate_Asian_V'] = d['Asian American']

        # new_d['Numb_AIndA'] = np.nan
        new_d['Rate_AIndA_V'] = d['American Indian or Alaska Native (AI/AN)']

        # new_d['Numb_Other'] = np.nan
        new_d['Rate_Other_V'] = d['Other']

        new_d['Date'] = d['Latest_Date']

        new_L.append(new_d)


    Data = pd.DataFrame(new_L)

    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)

    
    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')

        
        
    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    
    return T
    

    

CA_County_Vaccine_Race = get_California_Raw_and_postprocess(US_COUNTY_POPULATION)

# CA_County_Vaccine_Race.to_csv('FinalResult/Post_California.csv')
# print(CA_County_Vaccine_Race.shape)
CA_County_Vaccine_Race.head()

,County,Latest_Date,American Indian or Alaska Native (AI/AN),Asian American,Black,Latino,Multi-race,Native Hawaiian or Other Pacific Islander (NHPI),White,Other,Unknown
0,Alameda,2021-05-18,0.003037,0.322841,0.067144,0.139323,0.012683,0.007546,0.301939,0.086290,0.059195
1,Alpine,2021-05-17,0.017032,0.012165,0.002433,0.012165,0.000000,0.000000,0.192214,0.010949,0.753041
2,Amador,2021-05-18,0.008574,0.017790,0.005144,0.082789,0.001179,0.003644,0.739792,0.091041,0.050048
3,Butte,2021-05-18,0.012459,0.044084,0.010821,0.118308,0.013414,0.002051,0.703553,0.058818,0.036493
4,Calaveras,2021-05-18,0.005031,0.017243,0.005713,0.073899,0.000839,0.002411,0.619811,0.082075,0.192977


In [8]:
T = standardize_California(CA_County_Vaccine_Race, US_COUNTY_POPULATION)
state = 'California'

print(T.shape)
T.head()

(58, 18)


,State,County,Geographic Area Name,Date,Rate_AIndA_V,Rate_Asian_V,Rate_Black_V,Rate_Other_V,Rate_White_V,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,California,Alameda,"Alameda, California",2021-05-18,0.003037,0.322841,0.067144,0.086290,0.301939,17797,37797,540391,594001,184368,214392,823593,899377,1671329
1,California,Alpine,"Alpine, California",2021-05-17,0.017032,0.012165,0.002433,0.010949,0.192214,290,310,18,27,4,26,767,813,1129
2,California,Amador,"Amador, California",2021-05-18,0.008574,0.017790,0.005144,0.091041,0.739792,925,1644,665,1043,1066,1374,35636,36844,39752
3,California,Butte,"Butte, California",2021-05-18,0.012459,0.044084,0.010821,0.058818,0.703553,5547,9925,10984,14403,4158,7087,187742,197213,219186
4,California,Calaveras,"Calaveras, California",2021-05-18,0.005031,0.017243,0.005713,0.082075,0.619811,905,1858,859,1399,498,836,41752,43344,45905


In [9]:
not_included = list(set(T['Geographic Area Name'].unique()) - set(CalInfo['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)

    
not_included = list(set(CalInfo['Geographic Area Name'].unique()) - set(T['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


Not included Counties [nan, 'Unknown, Not in California', 'Outside California, Not in California']


In [10]:
T2 = pd.merge(T, CalInfo[['Geographic Area Name', 'cumulative_at_least_one_dose']] , on = 'Geographic Area Name')
print(T2.shape)
T2.head()

(58, 19)


,State,County,Geographic Area Name,Date,Rate_AIndA_V,Rate_Asian_V,Rate_Black_V,Rate_Other_V,Rate_White_V,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole,cumulative_at_least_one_dose
0,California,Alameda,"Alameda, California",2021-05-18,0.003037,0.322841,0.067144,0.086290,0.301939,17797,37797,540391,594001,184368,214392,823593,899377,1671329,1032515
1,California,Alpine,"Alpine, California",2021-05-17,0.017032,0.012165,0.002433,0.010949,0.192214,290,310,18,27,4,26,767,813,1129,823
2,California,Amador,"Amador, California",2021-05-18,0.008574,0.017790,0.005144,0.091041,0.739792,925,1644,665,1043,1066,1374,35636,36844,39752,18699
3,California,Butte,"Butte, California",2021-05-18,0.012459,0.044084,0.010821,0.058818,0.703553,5547,9925,10984,14403,4158,7087,187742,197213,219186,85244
4,California,Calaveras,"Calaveras, California",2021-05-18,0.005031,0.017243,0.005713,0.082075,0.619811,905,1858,859,1399,498,836,41752,43344,45905,19161


In [11]:
T2.columns

Index(['State', 'County', 'Geographic Area Name', 'Date', 'Rate_AIndA_V',
       'Rate_Asian_V', 'Rate_Black_V', 'Rate_Other_V', 'Rate_White_V',
       'Total_AIndA', 'Total_AIndAC', 'Total_Asian', 'Total_AsianC',
       'Total_Black', 'Total_BlackC', 'Total_White', 'Total_WhiteC',
       'Total_Whole', 'cumulative_at_least_one_dose'],
      dtype='object')

In [12]:
for Race in ['White', 'Black']:
    T2['Numb_' + Race]        = T2['cumulative_at_least_one_dose'] * T2['Rate_' + Race + '_V']
    T2['Rate_' + Race]       = T2['Numb_' + Race] / T2['Total_' + Race]
    # T2['Rate_' + Race + 'C'] = T2['Num_' + Race] / T2['Total_' + Race + 'C']

    
cols = ['State', 'County', 'Geographic Area Name', 'Date', 'Numb_White',
       'Rate_White', 'Numb_Black', 'Rate_Black',
       'Total_AIndA', 'Total_AIndAC', 'Total_Asian', 'Total_AsianC',
       'Total_Black', 'Total_BlackC', 'Total_White', 'Total_WhiteC',
       'Total_Whole', 'cumulative_at_least_one_dose', ]
T = T2[cols]
T.head()

,State,County,Geographic Area Name,Date,Numb_White,Rate_White,Numb_Black,Rate_Black,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole,cumulative_at_least_one_dose
0,California,Alameda,"Alameda, California",2021-05-18,311756.546585,0.378532,69327.187160,0.376026,17797,37797,540391,594001,184368,214392,823593,899377,1671329,1032515
1,California,Alpine,"Alpine, California",2021-05-17,158.192122,0.206248,2.002359,0.500590,290,310,18,27,4,26,767,813,1129,823
2,California,Amador,"Amador, California",2021-05-18,13833.370608,0.388185,96.187656,0.090232,925,1644,665,1043,1066,1374,35636,36844,39752,18699
3,California,Butte,"Butte, California",2021-05-18,59973.671932,0.319447,922.425324,0.221844,5547,9925,10984,14403,4158,7087,187742,197213,219186,85244
4,California,Calaveras,"Calaveras, California",2021-05-18,11876.198571,0.284446,109.466793,0.219813,905,1858,859,1399,498,836,41752,43344,45905,19161


In [13]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California']


<ipython-input-13-266ad6d67fc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_White,58.0,0.288675,0.110941,0.050642,0.189658,0.300151,0.378843,0.570614
1,Rate_Black,58.0,0.231785,0.104841,0.025853,0.161450,0.222143,0.310835,0.500590
2,Rate_Parity,58.0,0.056890,0.113748,-0.294342,-0.012783,0.033863,0.127926,0.338948


# III. Table States

## [!] B. Tennessee* (Done)


https://www.tn.gov/health/cedep/ncov/data/downloadable-datasets.html

Covid Vaccine County Demographics


95 counties





In [14]:
import pandas as pd

url = 'https://www.tn.gov/content/dam/tn/health/documents/cedep/novel-coronavirus/datasets/COVID_VACCINE_COUNTY_DEMOGRAPHICS.XLSX'
RawData = pd.read_excel(url)
RawData.tail()  # RawData

,DATE,COUNTY,CATEGORY,CAT_DETAIL,RECIPIENT_COUNT
127449,2021-05-20,WILSON,ETHN,UNKNOWN,6545.0
127450,2021-05-20,WILSON,SEX,F,28994.0
127451,2021-05-20,WILSON,SEX,M,23029.0
127452,2021-05-20,WILSON,SEX,O,46.0
127453,2021-05-20,WILSON,SEX,U,80.0


In [15]:
######################################################
Date = Today_Date 
######################################################  


# RawData2 = RawData

while (RawData['DATE'] == Date).sum() == 0:
    print('There is no information for current Date:', Date)
    Date = input('Please Enter a new One:')

    
    

print('We get Tennessee Information for Date:', Date)
RawData = RawData[RawData['DATE'] == Date]

RawData.head()

We get Tennessee Information for Date: 2021-05-20


,DATE,COUNTY,CATEGORY,CAT_DETAIL,RECIPIENT_COUNT
126317,2021-05-20,ANDERSON,RACE,ASIAN,364.0
126318,2021-05-20,ANDERSON,RACE,BLACK OR AFRICAN AMERICAN,697.0
126319,2021-05-20,ANDERSON,RACE,OTHER/MULTIRACIAL,5373.0
126320,2021-05-20,ANDERSON,RACE,UNKNOWN,2705.0
126321,2021-05-20,ANDERSON,RACE,WHITE,23039.0


In [16]:


RawData = RawData[RawData['CATEGORY'].isin(['RACE', 'ETHN'])]


L= []

for county, record in RawData.groupby('COUNTY'):
    # print(county)
    D = {}
    D['County'] = county

    for idx, row in record.iterrows():
        d = row.to_dict()
        # print(d)
        
        item = d['CAT_DETAIL']
        if item == 'UNKNOWN': 
            item = d['CATEGORY'] + '-' + item
        
        D[item] = d['RECIPIENT_COUNT']
    # print(D)
    L.append(D)

Tennessee_County_Vaccine_Race = pd.DataFrame(L)


In [17]:

# Tennessee_County_Vaccine_Race.to_csv('FinalResult/Tennessee.csv')
print(Tennessee_County_Vaccine_Race.shape)
Tennessee_County_Vaccine_Race.head() # 95 counties in Tennessee

(96, 9)


,County,ASIAN,BLACK OR AFRICAN AMERICAN,OTHER/MULTIRACIAL,RACE-UNKNOWN,WHITE,HISPANIC OR LATINO,NOT HISPANIC OR LATINO,ETHN-UNKNOWN
0,ANDERSON,364.0,697.0,5373.0,2705.0,23039.0,960.0,27150.0,4068.0
1,BEDFORD,139.0,785.0,1897.0,1025.0,8982.0,968.0,10376.0,1484.0
2,BENTON,40.0,229.0,611.0,129.0,4619.0,93.0,5274.0,261.0
3,BLEDSOE,NaN,47.0,436.0,321.0,3002.0,56.0,3241.0,511.0
4,BLOUNT,387.0,824.0,6265.0,3370.0,40292.0,967.0,44699.0,5472.0


In [18]:
# print(len(L))

import numpy as np



state = 'Tennessee' ## 

new_L = []


for idx, row in Tennessee_County_Vaccine_Race.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        'Dekalb': 'DeKalb',
        'Mcminn': 'McMinn', 
        'Mcnairy': 'McNairy',
        # 'Mccormick': 'McCormick'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
        
        
    new_d['Numb_White'] = d['WHITE']
    
    new_d['Numb_Black'] = d['BLACK OR AFRICAN AMERICAN']
    
    
    new_d['Numb_Asian'] = d['ASIAN']
    new_d['Rate_Asian'] = np.nan
    
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]


print(T.shape)
T.head()

Not included Counties ['Out Of State, Tennessee']
(95, 19)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_White,Rate_Asian,Rate_Black,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Tennessee,Anderson,"Anderson, Tennessee",NaN,364.0,697.0,23039.0,0.310580,0.220989,0.326906,371,911,1172,1480,3154,4161,70476,72149,76978
1,Tennessee,Bedford,"Bedford, Tennessee",NaN,139.0,785.0,8982.0,0.267308,0.186949,0.207662,580,843,520,669,4199,4922,43253,44280,49713
2,Tennessee,Benton,"Benton, Tennessee",NaN,40.0,229.0,4619.0,0.408163,0.546539,0.302152,91,220,98,136,419,536,15287,15532,16160
3,Tennessee,Bledsoe,"Bledsoe, Tennessee",NaN,NaN,47.0,3002.0,NaN,0.040517,0.221599,96,188,45,71,1160,1261,13547,13750,15064
4,Tennessee,Blount,"Blount, Tennessee",NaN,387.0,824.0,40292.0,0.297692,0.204975,0.322951,564,1487,1300,1754,4020,5075,124762,126972,133088


In [19]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,90.0,0.232856,0.119754,0.040517,0.171909,0.209536,0.257416,0.863014
1,Rate_White,95.0,0.276508,0.058253,0.146253,0.238588,0.264556,0.303104,0.457310
2,Rate_Parity,90.0,0.046551,0.115989,-0.546870,0.015471,0.063743,0.108698,0.196795


## [!] B. North Carolina* (Done)

Download the Data First

https://covid19.ncdhhs.gov/dashboard/data-behind-dashboards

In [30]:
'CountyVaccine/TMP/May20/NorthCarolina/Vaccination_Race-Fed-Cnty.xlsx'


'CountyVaccine/TMP/May20/NorthCarolina/Vaccination_Race-Fed-Cnty.xlsx'

In [31]:

Date = Today_Date
import os

while True:
    Path = input('Enter North Carolina Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)


FirstData = pd.read_excel(Path)
FirstData.columns



Enter North Carolina Manually collected Data:  CountyVaccine/TMP/May20/NorthCarolina/Vaccination_Race-Fed-Cnty.xlsx
File Exists!
Enter "Yes" if the file contains the information of the date you wanted:  Yes

The Path we have:
CountyVaccine/TMP/May20/NorthCarolina/Vaccination_Race-Fed-Cnty.xlsx


Index(['Index', 'County ', 'Week of', 'American Indian or Alaskan Native',
       'Asian or Pacific Islander', 'Black or African American', 'White',
       'Other', 'Suppressed', 'Missing or Undisclosed'],
      dtype='object')

In [32]:

Date = Today_Date
import os

while True:
    Path = input('Enter North Carolina Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)


FullData = pd.read_excel(Path)
FullData.columns



Enter North Carolina Manually collected Data:  CountyVaccine/TMP/May20/NorthCarolina/Vaccination_Race-Fed-Cnty-full.xlsx
File Exists!
Enter "Yes" if the file contains the information of the date you wanted:  Yes

The Path we have:
CountyVaccine/TMP/May20/NorthCarolina/Vaccination_Race-Fed-Cnty-full.xlsx


Index(['Index', 'County ', 'Week of', 'American Indian or Alaskan Native',
       'Asian or Pacific Islander', 'Black or African American', 'White',
       'Other', 'Suppressed', 'Missing or Undisclosed'],
      dtype='object')

In [34]:

columns = ['County ', 'Week of', 'American Indian or Alaskan Native',
       'Asian or Pacific Islander', 'Black or African American',
       'Missing or Undisclosed', 'Other', 'Suppressed', 'White']


NorthCarolina = DataFirst[columns]
Groups = NorthCarolina.groupby('County ')

CL = []


for County, df in Groups:
    D = {}
    D['County'] = County
    d = df.sum().to_dict() # Here is a sum
    D['Black or African American'] = d['Black or African American']
    D['American Indian or Alaskan Native'] = d['American Indian or Alaskan Native']
    D['White'] = d['White']
    D['Asian or Pacific Islander'] = d['Asian or Pacific Islander']
    
    CL.append(D)
    
NCFirst = pd.DataFrame(CL)
NCFirst = NCFirst.replace(0.0, np.nan) ### Changes here



NorthCarolina = FullData[columns]
Groups = NorthCarolina.groupby('County ')

CL = []
for County, df in Groups:
    D = {}
    D['County'] = County
    d = df.sum().to_dict() # Here is a sum
    D['Black or African American-Full'] = d['Black or African American']
    D['American Indian or Alaskan Native-Full'] = d['American Indian or Alaskan Native']
    D['White-Full'] = d['White']
    D['Asian or Pacific Islander-Full'] = d['Asian or Pacific Islander']
    CL.append(D)
    
NCFull = pd.DataFrame(CL)
NCFull = NCFull.replace(0.0, np.nan) ### Changes here


NC = NCFirst.merge(NCFull, on = 'County')
NC

,County,Black or African American,American Indian or Alaskan Native,White,Asian or Pacific Islander,Black or African American-Full,American Indian or Alaskan Native-Full,White-Full,Asian or Pacific Islander-Full
0,Alamance,1590.0,61.0,6760.0,300.0,1218.0,73.0,5980.0,244.0
1,Alexander,38.0,NaN,756.0,NaN,38.0,NaN,665.0,NaN
2,Alleghany,NaN,NaN,659.0,NaN,NaN,NaN,588.0,NaN
3,Anson,211.0,NaN,292.0,NaN,161.0,NaN,237.0,NaN
4,Ashe,NaN,NaN,1426.0,NaN,NaN,NaN,1320.0,NaN
...,...,...,...,...,...,...,...,...,...
97,Wayne,1732.0,37.0,2981.0,174.0,1453.0,35.0,2555.0,128.0
98,Wilkes,154.0,NaN,2969.0,NaN,111.0,NaN,2362.0,NaN
99,Wilson,1644.0,32.0,2249.0,170.0,1426.0,31.0,1964.0,138.0
100,Yadkin,58.0,NaN,1310.0,NaN,58.0,NaN,1201.0,NaN


In [35]:
# print(len(L))

import numpy as np

state = 'North Carolina' ## 

new_L = []


for idx, row in NC.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
    if type(d['County']) != str:
        print(d)
        continue 
        
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
        'Mcdowell': 'McDowell'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
    
    
    new_d['Numb_White'] = d['White']
    
    new_d['Numb_Black'] = d['Black or African American']
    
    new_d['Numb_Asian'] = d['Asian or Pacific Islander']
    
    new_d['Numb_AIndA'] = d['American Indian or Alaskan Native']
    
    
    new_d['FullNumb_White'] = d['White-Full']
    
    new_d['FullNumb_Black'] = d['Black or African American-Full']
    
    new_d['FullNumb_Asian'] = d['Asian or Pacific Islander-Full']
    
    new_d['FullNumb_AIndA'] = d['American Indian or Alaskan Native-Full']
    
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate
    Rate = T['FullNumb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['FullRate_' + Race] = Rate

#     Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
#     Rate = Rate.replace([-np.inf, np.inf], np.nan)
#     T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]

print(T.shape)
T.head()

Not included Counties ['Out Of State, North Carolina', 'Missing, North Carolina']
(100, 27)


,State,County,Geographic Area Name,Date,FullNumb_AIndA,FullNumb_Asian,FullNumb_Black,FullNumb_White,FullRate_Asian,FullRate_Black,...,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,North Carolina,Alamance,"Alamance, North Carolina",NaN,73.0,244.0,1218.0,5980.0,0.082488,0.034437,...,0.054212,2466,3610,2958,3794,35369,37943,124696,128147,169509
1,North Carolina,Alexander,"Alexander, North Carolina",NaN,NaN,NaN,38.0,665.0,NaN,0.017164,...,0.022205,199,376,385,470,2214,2630,34047,34625,37497
2,North Carolina,Alleghany,"Alleghany, North Carolina",NaN,NaN,NaN,NaN,588.0,NaN,NaN,...,0.061994,59,144,89,107,197,269,10630,10766,11137
3,North Carolina,Anson,"Anson, North Carolina",NaN,NaN,NaN,161.0,237.0,NaN,0.013591,...,0.025198,227,368,360,419,11846,12094,11588,11960,24446
4,North Carolina,Ashe,"Ashe, North Carolina",NaN,NaN,NaN,NaN,1320.0,NaN,NaN,...,0.054114,114,269,138,176,262,416,26352,26643,27203


In [36]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['FullRate_Parity']  = T['FullRate_White'] - T['FullRate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee', 'North Carolina']


,name,count,mean,std,min,25%,50%,75%,max
0,FullRate_Black,78.0,0.035986,0.012234,0.013591,0.028193,0.036004,0.041853,0.076923
1,FullRate_White,97.0,0.043566,0.021088,0.009223,0.028549,0.039678,0.054595,0.109376
2,Rate_Black,78.0,0.043876,0.015192,0.013072,0.033490,0.044344,0.050431,0.095272
3,Rate_White,97.0,0.049925,0.023384,0.010101,0.034352,0.046812,0.061616,0.122033
4,Rate_Parity,75.0,0.011277,0.016732,-0.011943,-0.000102,0.006590,0.016734,0.062283
5,FullRate_Parity,75.0,0.011952,0.016571,-0.011192,0.000828,0.007348,0.015933,0.058177


# IV. OCR States

## [!] C. West Virginia* (Done)


Copy Response from Browsers

https://dhhr.wv.gov/COVID-19/Pages/default.aspx

url = 'https://wabi-us-gov-virginia-api.analysis.usgovcloudapi.net/public/reports/querydata?synchronous=true'




In [61]:
'CountyVaccine/TMP/May20/WestVirginia/WV5_20.json'
'CountyVaccine/TMP/May20/WestVirginia/WV5_20-full.json'

'CountyVaccine/TMP/May20/WestVirginia/WV5_20-full.json'

In [68]:


Date = Today_Date
import os

while True:
    Path = input('Enter West Virginia Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
        
print('\nThe Path we have:')
print(Path)


Enter West Virginia Manually collected Data:  CountyVaccine/TMP/May20/WestVirginia/WV5_20.json
File Exists!
Enter "Yes" if the file contains the information of the date you wanted:  Yes

The Path we have:
CountyVaccine/TMP/May20/WestVirginia/WV5_20.json


In [69]:

import requests
import json


date = Today_Date


def get_WestVirginia_Raw(date, response_data, prefix = ''):
    

    # data = json.loads(response_data)
    data = response_data

    tmp = data['results'][0]['result']['data']['dsr']['DS'][0]
    # [i for i in tmp]

    tmp2 = tmp['PH'][1]['DM1']
    # len(tmp2)
    # tmp2


    L = []
    for county_info in tmp2:
        d= {}
        d['County'] = county_info['G0']
        d['Date' ] =  date
        try:
            info = [race['C'] for race in county_info['X'][:-1]]
            # print(info)

            d[prefix + 'Numb_Black'] = info[0][0]
            d[prefix + 'Numb_White'] = info[-1][0]
            L.append(d)
        except:
            print(d['County'])
            print(county_info['X'])


    WestVirginia = pd.DataFrame(L)
    return WestVirginia
    
    
    
with open(Path, 'r') as f:
    response_data = json.load(f)
    
    


    
    
WestVirginiaFirst = get_WestVirginia_Raw(date, response_data)

WestVirginiaFirst

,County,Date,Numb_Black,Numb_White
0,Barbour,2021-05-20,21,4366
1,Berkeley,2021-05-20,2337,28453
2,Boone,2021-05-20,48,6547
3,Braxton,2021-05-20,16,5061
4,Brooke,2021-05-20,54,7285
5,Cabell,2021-05-20,1422,35776
6,Calhoun,2021-05-20,1,2134
7,Clay,2021-05-20,6,2844
8,Doddridge,2021-05-20,26,3627
9,Fayette,2021-05-20,702,16688


In [70]:
Date = Today_Date
import os

while True:
    Path = input('Enter West Virginia Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
FullPath = Path   
print('\nThe Path we have:')
print(FullPath)


Enter West Virginia Manually collected Data:  CountyVaccine/TMP/May20/WestVirginia/WV5_20-full.json
File Exists!
Enter "Yes" if the file contains the information of the date you wanted:  Yes

The Path we have:
CountyVaccine/TMP/May20/WestVirginia/WV5_20-full.json


In [71]:
with open(FullPath, 'r') as f:
    response_data_full = json.load(f)


WestVirginiaFull = get_WestVirginia_Raw(date, response_data_full, prefix = 'Full')
# WestVirginia.to_csv('FinalResult/Post_WestVirginia.csv')
WestVirginiaFull

,County,Date,FullNumb_Black,FullNumb_White
0,Barbour,2021-05-20,18,3554
1,Berkeley,2021-05-20,1858,23690
2,Boone,2021-05-20,38,5316
3,Braxton,2021-05-20,14,4172
4,Brooke,2021-05-20,48,6545
5,Cabell,2021-05-20,1157,30473
6,Calhoun,2021-05-20,1,1637
7,Clay,2021-05-20,3,2361
8,Doddridge,2021-05-20,20,3280
9,Fayette,2021-05-20,621,14431


In [72]:

WestVirginia = pd.merge(WestVirginiaFirst, WestVirginiaFull, on = 'County')
print(WestVirginia.shape)
WestVirginia.head() # 55 counties


(55, 7)


,County,Date_x,Numb_Black,Numb_White,Date_y,FullNumb_Black,FullNumb_White
0,Barbour,2021-05-20,21,4366,2021-05-20,18,3554
1,Berkeley,2021-05-20,2337,28453,2021-05-20,1858,23690
2,Boone,2021-05-20,48,6547,2021-05-20,38,5316
3,Braxton,2021-05-20,16,5061,2021-05-20,14,4172
4,Brooke,2021-05-20,54,7285,2021-05-20,48,6545


In [73]:
# print(len(L))

import numpy as np

state = 'West Virginia' ## 

new_L = []

for idx, row in WestVirginia.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
        
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
     'Mcdowell': 'McDowell'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

    new_d['Numb_White'] = d['Numb_White']
    # new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Numb_Black']
    
    
    new_d['FullNumb_White'] = d['FullNumb_White']
    # new_d['Rate_White'] = np.nan
    
    new_d['FullNumb_Black'] = d['FullNumb_Black']
    
    
    # new_d['Date'] = d['Date']
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)


not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate
    
    Rate = T['FullNumb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['FullRate_' + Race] = Rate



T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]

print(T.shape)
T.head()

(55, 20)


,State,County,Geographic Area Name,FullNumb_Black,FullNumb_White,FullRate_Black,FullRate_White,Numb_Black,Numb_White,Rate_Black,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,West Virginia,Barbour,"Barbour, West Virginia",18,3554,0.108434,0.224072,21,4366,0.126506,0.275266,106,225,50,91,166,280,15861,16103,16441
1,West Virginia,Berkeley,"Berkeley, West Virginia",1858,23690,0.197660,0.226725,2337,28453,0.248617,0.272309,300,1074,1328,2011,9400,11814,104488,107962,119171
2,West Virginia,Boone,"Boone, West Virginia",38,5316,0.238994,0.252362,48,6547,0.301887,0.310800,33,95,27,41,159,260,21065,21228,21457
3,West Virginia,Braxton,"Braxton, West Virginia",14,4172,0.141414,0.307126,16,5061,0.161616,0.372571,61,135,33,62,99,178,13584,13751,13957
4,West Virginia,Brooke,"Brooke, West Virginia",48,6545,0.147692,0.309676,54,7285,0.166154,0.344689,41,148,110,157,325,516,21135,21444,21939


In [74]:
# West Virginia	Barbour	Barbour, West Virginia	2021-03-26	10	2839	0.060241	0.035714	0.178992	0.176303	106	225	50

T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['FullRate_Parity']  = T['FullRate_White'] - T['FullRate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee', 'North Carolina', 'West Virginia']


,name,count,mean,std,min,25%,50%,75%,max
0,FullRate_Black,55.0,0.186924,0.093325,0.015366,0.115246,0.187739,0.259972,0.379310
1,FullRate_White,55.0,0.316548,0.054320,0.208516,0.285238,0.309981,0.352190,0.447993
2,Rate_Black,55.0,0.227316,0.106432,0.020095,0.142190,0.230769,0.311304,0.418502
3,Rate_White,55.0,0.363615,0.056346,0.246356,0.328596,0.359165,0.402854,0.492912
4,Rate_Parity,55.0,0.136299,0.096159,-0.053133,0.079568,0.130063,0.208274,0.326974
5,FullRate_Parity,55.0,0.129623,0.084744,-0.043269,0.080867,0.121919,0.196869,0.300573


# VI. Manual States

## [!] E. Maine* (Done)

In [77]:
'CountyVaccine/TMP/May20/Maine/Maine 5_20.xlsx'

'CountyVaccine/TMP/May20/Maine/Maine 5_20.xlsx'

In [78]:
import os


Date = Today_Date


while True:
    Path = input('Enter Maine Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)


Enter Maine Manually collected Data:  CountyVaccine/TMP/May20/Maine/Maine 5_20.xlsx
File Exists!
Enter "Yes" if the file contains the information of the date you wanted:  Yes

The Path we have:
CountyVaccine/TMP/May20/Maine/Maine 5_20.xlsx


In [84]:

try:
    Data = pd.read_csv(Path)
except:
    Data = pd.read_excel(Path)
    
Data = Data[Data.columns[1:]]
Maine = Data
# Maine.to_csv('FinalResult/Maine.csv')
Maine = Maine.rename(columns = {'Black or African American': 'Black'})
Maine

,County,Black,White,Black-full,White-full
0,ANDROSCOGGIN,1193,36828,852,33960
1,AROOSTOOK,135,22121,115,22853
2,CUMBERLAND,3659,141819,2723,132266
3,FRANKLIN,39,10803,25,10761
4,HANCOCK,216,24623,151,23906
5,KENNEBEC,394,44858,298,42332
6,KNOX,109,18511,78,18425
7,LINCOLN,81,17604,58,16441
8,OXFORD,112,21604,86,20233
9,PENOBSCOT,590,58142,518,56571


In [86]:
# print(len(L))

import numpy as np
state = 'Maine' ## 

new_L = []
for idx, row in Maine.iterrows(): ## 
    d = row.to_dict()
    
    # print(d)
    
    new_d = {}
    
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
    
    new_d['Numb_White'] = d['White'] if d['White']!= '-' else np.nan
    # new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Black'] if d['Black']!= '-' else np.nan
    # new_d['Rate_Black'] = np.nan
    
    
    new_d['FullNumb_White'] = d['White-full'] if d['White-full']!= '-' else np.nan
    # new_d['Rate_White'] = np.nan
    
    new_d['FullNumb_Black'] = d['Black-full'] if d['Black-full']!= '-' else np.nan
    # new_d['Rate_Black'] = np.nan

    
    # new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate
    
    Rate = T['FullNumb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['FullRate_' + Race] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
print(T.shape)
T.head()

(16, 20)


,State,County,Geographic Area Name,FullNumb_Black,FullNumb_White,FullRate_Black,FullRate_White,Numb_Black,Numb_White,Rate_Black,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Maine,Androscoggin,"Androscoggin, Maine",852,33960,0.174877,0.341949,1193,36828,0.244869,0.370828,457,1251,1014,1685,4872,6145,99313,101811,108277
1,Maine,Aroostook,"Aroostook, Maine",115,22853,0.144472,0.359510,135,22121,0.169598,0.347995,1293,1922,343,516,796,1063,63567,64576,67055
2,Maine,Cumberland,"Cumberland, Maine",2723,132266,0.288118,0.487360,3659,141819,0.387155,0.522560,1037,2945,7057,9344,9451,11622,271393,277223,295003
3,Maine,Franklin,"Franklin, Maine",25,10761,0.149701,0.368288,39,10803,0.233533,0.369725,146,440,160,259,167,299,29219,29696,30199
4,Maine,Hancock,"Hancock, Maine",151,23906,0.266314,0.453891,216,24623,0.380952,0.467505,261,664,684,878,567,805,52669,53436,54987


In [88]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['FullRate_Parity']  = T['FullRate_White'] - T['FullRate_Black']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee', 'North Carolina', 'West Virginia', 'Maine']


,name,count,mean,std,min,25%,50%,75%,max
0,FullRate_Black,16.0,0.232383,0.069406,0.096154,0.187682,0.245216,0.279520,0.346952
1,FullRate_White,16.0,0.400520,0.052156,0.335752,0.362237,0.380987,0.430221,0.490498
2,Rate_Black,16.0,0.303934,0.099100,0.134615,0.227781,0.331019,0.382503,0.465257
3,Rate_White,16.0,0.408698,0.069694,0.293909,0.370552,0.395338,0.464380,0.525195
4,Rate_Parity,16.0,0.104764,0.069486,-0.014566,0.072174,0.117208,0.141047,0.248012
5,FullRate_Parity,16.0,0.168137,0.063326,0.045754,0.139952,0.169023,0.215925,0.273114


## [!]  E. New Jersey* (Done)

In [98]:
'CountyVaccine/TMP/May20/NewJersey/New Jersey 5_20.xlsx'

'CountyVaccine/TMP/May20/NewJersey/New Jersey 5_20.xlsx'

In [99]:
import os

Date = Today_Date


while True:
    Path = input('Enter **New Jersey** Manually collected Data:  ')
    if os.path.isfile(Path):
        print('File Exists!')
        x = input('Enter "Yes" if the file contains the information of the date you wanted:  ')
        if x =='Yes':
            break
    else:
        print('Wrong file path...')
        
        
print('\nThe Path we have:')
print(Path)


Enter **New Jersey** Manually collected Data:  CountyVaccine/TMP/May20/NewJersey/New Jersey 5_20.xlsx
File Exists!
Enter "Yes" if the file contains the information of the date you wanted:  Yes

The Path we have:
CountyVaccine/TMP/May20/NewJersey/New Jersey 5_20.xlsx


In [100]:
try:
    Data = pd.read_csv(Path)
except:
    Data = pd.read_excel(Path)
    
if len(Data.columns) == 6:
    Data = Data[Data.columns[1:]]


print(Data.columns)

# Data.columns = ['State full', 'County', 'People with at least one vaccine dose','Black', 'White']
NewJersey = Data

# NewJersey.to_csv('FinalResult/NewJersey.csv')

NewJersey.head()

Index(['State full', 'County', 'Black-full', 'White-full'], dtype='object')


,State full,County,Black-full,White-full
0,New Jersey,ATLANTIC,19667,135807
1,New Jersey,BERGEN,28169,447742
2,New Jersey,BURLINGTON,25252,281348
3,New Jersey,CAMDEN,42832,273819
4,New Jersey,CAPE MAY,1900,74425


In [101]:
# NewJersey[['Black', 'White'] ].sum(axis=1)

In [102]:
# print(len(L))

import numpy as np

state = 'New Jersey' ## 

new_L = []


for idx, row in NewJersey.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
    
    # new_d['Numb_White'] = d['People with at least one vaccine dose'] *  d['White'] / 100
    # new_d['Numb_Black'] = d['People with at least one vaccine dose'] *  d['Black'] / 100
    
    new_d['FullNumb_White'] = d['White-full']
    new_d['FullNumb_Black'] = d['Black-full']
    
    
    # new_d['Numb_Asian'] = d['First Dose Total'] *  d['Asian %'] / 100ba wo
    # new_d['Numb_Hispa'] = d['First Dose Total'] * d['Hispanic/Latinx %'] / 100
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black']:
    Rate = T['FullNumb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['FullRate_' + Race] = Rate

T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]


print(T.shape)
T.head()

(21, 17)


,State,County,Geographic Area Name,Date,FullNumb_Black,FullNumb_White,FullRate_Black,FullRate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,New Jersey,Atlantic,"Atlantic, New Jersey",NaN,19667,135807,0.435294,0.722443,1782,3466,21282,23764,45181,49558,187983,194066,263670
1,New Jersey,Bergen,"Bergen, New Jersey",NaN,28169,447742,0.411040,0.659121,4508,8451,158020,168160,68531,78683,679302,697546,932202
2,New Jersey,Burlington,"Burlington, New Jersey",NaN,25252,281348,0.309783,0.867057,1520,4758,24118,28721,81515,90134,324486,335601,445349
3,New Jersey,Camden,"Camden, New Jersey",NaN,42832,273819,0.391010,0.783376,3304,6565,30391,34188,109542,118445,349537,360646,506471
4,New Jersey,Cape May,"Cape May, New Jersey",NaN,1900,74425,0.426679,0.881322,336,723,943,1366,4453,5514,84447,86071,92039


In [103]:
T['FullRate_Parity']  = T['FullRate_White'] - T['FullRate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['California', 'Tennessee', 'North Carolina', 'West Virginia', 'Maine', 'New Jersey']


,name,count,mean,std,min,25%,50%,75%,max
0,FullRate_Black,21.0,0.423625,0.090209,0.259612,0.371882,0.418672,0.450308,0.654130
1,FullRate_White,21.0,0.684364,0.139186,0.360144,0.608698,0.703249,0.770550,0.881322
2,FullRate_Parity,21.0,0.260740,0.111197,0.049982,0.184595,0.260791,0.287149,0.557274


# All in One

## Read Previous Vaccine Data

In [37]:
# import os
# path = ''

# print('Input Historical STATE_VACCINE')
# print('For example: RaceVaccineBackup/STATE_VACCINE_2021-03-23.p')

# L = []
# while True:
    
#     path = input()
    
#     if os.path.isfile(path):
#         print('Path Exists!')
#         L.append(path)
#     elif path != '':
#         print('Wrong Path')
#     else:
#         break

# print('End')

In [38]:
# import pickle

# Historical_Path = {i.split('_')[-1].replace('.p', ''): i for i in L }

# Historical_Data = {}
# for date, path in Historical_Path.items():
#     with open(path, 'rb' ) as f:
#         Historical_Data[date] = pickle.load(f)
    
    
# [i for i in Historical_Data]

## Show Parity for Each State

In [39]:
# ######################################

# print(Today_Date)
# ######################################

# for state in STATE_VACCINE:
    
#     print(state)
#     cols = ['name',  'date', 'count', 'mean', 'min','50%', 'max']

    
#     for date, Old_STATE_VACCINE in Historical_Data.items():
#         x = diagnose_Rate(Old_STATE_VACCINE[state], state = state, date = date)
#         print(x[cols])
    
#     x = diagnose_Rate(STATE_VACCINE[state], state = state, date = Today_Date)
    
#     print(x[cols])
#     print('\n')
    
    
# # pd.concat(L).to_csv('compare.csv')

##  Show Some Specials Cases

In [40]:
# # state = 'Texas'
# DF = pd.concat([v for k,v in STATE_VACCINE.items()]) 

# data_cols = ['State', 'County', 'Geographic Area Name',
#              'Total_Black', 'Total_White',
#              'Numb_Black', 'Numb_White','Rate_Black','Rate_White', 'Total_Whole', 'Rate_Parity']

# DF[DF['Rate_Black'] > 0.3][data_cols]

## Save Data

In [104]:

import pickle 

path = 'CountyVaccine/Workplace/STATE_VACCINE_' + Today_Date + '_Manual.p'

while True:
    print('The data will be saved here, do you agree?' )
    print(path)
    print('If yes, please enter: Yes')
    x = input()
    if x =='Yes':
        break


with open(path, 'wb') as f:
    pickle.dump(STATE_VACCINE, f)
    
    
print('Successfully Saved!')

The data will be saved here, do you agree?
CountyVaccine/Workplace/STATE_VACCINE_2021-05-20_Manual.p
If yes, please enter: Yes
Yes
Successfully Saved!


# Merge Data

In [105]:
def merge_data(date):
    D = {}
    for suffix in ['_Manual.p', '_Tableau.p', '_Autonomy.p']:
        path = 'CountyVaccine/Workplace/STATE_VACCINE_' + date + suffix
        print(path)
        with open(path, 'rb') as f:
            s = pickle.load(f)
            
        for k, v in s.items():
            D[k] = v
            
    return D
    
date = Today_Date
path = 'STATE_VACCINE_' + date + '.p'
D = merge_data(date)


CountyVaccine/Workplace/STATE_VACCINE_2021-05-20_Manual.p
CountyVaccine/Workplace/STATE_VACCINE_2021-05-20_Tableau.p
CountyVaccine/Workplace/STATE_VACCINE_2021-05-20_Autonomy.p


In [106]:
STATE_VACCINE = D

with open(path, 'wb') as f:
    pickle.dump(STATE_VACCINE, f)

print('save data to:', path)


save data to: STATE_VACCINE_2021-05-20.p


In [107]:
[i for i in STATE_VACCINE]

['California',
 'Tennessee',
 'North Carolina',
 'West Virginia',
 'Maine',
 'New Jersey',
 'New York',
 'Ohio',
 'Oregon',
 'South Carolina',
 'Wisconsin',
 'Illinois',
 'Texas',
 'Pennsylvania',
 'Indiana',
 'Virginia']